In [1]:
!which python


/Users/saiha/opt/anaconda3/envs/churn-model/bin/python


In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
#standard imports
import os
import pickle
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,cross_val_score
import sklearn

from src.localpath import *
from src.data.make_dataset import load_training_data
from src.models.train_model import store_model_and_results
from sklearn.linear_model import LogisticRegression


In [93]:
#Configuration for Jupyter Notebooks
pd.options.display.max_columns=100
pd.options.display.max_rows=100

## Load Featurized  Data

In [94]:
X_train,y_train=load_training_data(final=True)

In [95]:
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,0,1,1,0,1,25.30,25.30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,0,0,7,1,1,75.15,525.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,0,0,1,4,1,1,20.05,85.50,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,0,0,29,1,1,76.00,2215.25,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,0,0,0,3,1,1,75.10,270.70,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Set up the modeling pipeline
1. Train the model
2. Evaluate how good the model is(Looking at the results)
3. Pickle the Modeled Data
4. Save the model results to csv to track how each of our models perform

In [96]:
X,X_Validate,y,y_Validate =train_test_split(X_train,y_train, test_size=0.25,random_state=42)

In [97]:
X.shape


(3961, 30)

In [98]:
y.shape

(3961, 1)

# Set up the model

In [99]:
model=LogisticRegression()

In [100]:
model.fit(X,y)

/Users/saiha/opt/anaconda3/envs/churn-model/lib/python3.8/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/saiha/opt/anaconda3/envs/churn-model/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [101]:
#Model accuracy manual
np.mean((model.predict(X_Validate)==y_Validate.values[:,0]).astype(int))

0.7956093868281605

In [102]:
#Model accuracy Scikit Learn
model.score(X_Validate,y_Validate)

0.7956093868281605

# Pickle our model

In [103]:
model_filename=str(hash(np.random.rand())) + '.pkl'
model_filename

'964380814332921088.pkl'

In [104]:
with open (os.path.join(MODELS_DIRECTORY,model_filename),"wb") as f:
    pickle.dump(model,f)

# Save model results to CSV file

In [105]:
data_to_save={
    "model_filename":[model_filename],
    "model_string": [str(model)],
    "accuracy": [model.score(X_Validate,y_Validate)]
}

In [106]:
data_to_save

{'model_filename': ['964380814332921088.pkl'],
 'model_string': ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=100,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,\n                   warm_start=False)"],
 'accuracy': [0.7956093868281605]}

In [107]:
df_results= pd.DataFrame(data_to_save)

In [116]:
model_results_filename="model_results.csv"
df_results.to_csv(os.path.join(MODELS_DIRECTORY,model_results_filename),index=False)
df_results_test=pd.read_csv(os.path.join(MODELS_DIRECTORY,model_results_filename))
df_results_test

,model_filename,model_string,accuracy
0,964380814332921088.pkl,"LogisticRegression(C=1.0, class_weight=None, d...",0.795609


 # test our model storing function

In [115]:
store_model_and_results(model,X_train,y_train)

Saving model to 1074991767840951552.pkl
writing model results to existing results csv file


/Users/saiha/opt/anaconda3/envs/churn-model/lib/python3.8/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/saiha/opt/anaconda3/envs/churn-model/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Cross Validation

In [129]:
model=LogisticRegression(max_iter=1000)
print(np.mean(cross_val_score(model,X_train,y_train["Churn"],scoring="accuracy")))


0.8019663283736131


In [123]:
print("These are sklearn evaluation metrics")
for k in sklearn.metrics.SCORERS.keys():
    print(k)


These are sklearn evaluation metrics
explained_variance
r2
max_error
neg_median_absolute_error
neg_mean_absolute_error
neg_mean_squared_error
neg_mean_squared_log_error
neg_root_mean_squared_error
neg_mean_poisson_deviance
neg_mean_gamma_deviance
accuracy
roc_auc
roc_auc_ovr
roc_auc_ovo
roc_auc_ovr_weighted
roc_auc_ovo_weighted
balanced_accuracy
average_precision
neg_log_loss
neg_brier_score
adjusted_rand_score
homogeneity_score
completeness_score
v_measure_score
mutual_info_score
adjusted_mutual_info_score
normalized_mutual_info_score
fowlkes_mallows_score
precision
precision_macro
precision_micro
precision_samples
precision_weighted
recall
recall_macro
recall_micro
recall_samples
recall_weighted
f1
f1_macro
f1_micro
f1_samples
f1_weighted
jaccard
jaccard_macro
jaccard_micro
jaccard_samples
jaccard_weighted


# Test new model- storing function with additional metrics

In [131]:
model=LogisticRegression(max_iter=1000)
store_model_and_results(model,X_train,y_train)


fitting model before pickling
Saving model to 757674756096551424.pkl
writing model results to existing results csv file


/Users/saiha/opt/anaconda3/envs/churn-model/lib/python3.8/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
